In [ ]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
import string
import nltk
nltk.download('punkt')
import re

def preprocess_question(df, raw):
  # string cleaning function
  def clean_word(word):
    word = str(word)
    # lowercase
    word = word.lower()
    word = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", word)
    # cleaning operators
    word = re.sub(r"\/", " ", word)
    word = re.sub(r"\^", " ^ ", word)
    word = re.sub(r"\+", " + ", word)
    word = re.sub(r"\-", "", word)
    word = re.sub(r"\=", " = ", word)
    # translating abbreviations
    word = re.sub(r"\'re", 'are', word)
    word = re.sub(r"\'ve", 'have', word)
    word = re.sub(r"\'t", 'not', word)
    word = re.sub(r"\'d", 'would', word)
    word = re.sub(r"\'m", 'am', word)
    word = re.sub("where's", 'where is', word)
    word = re.sub("what's", 'what is', word)
    word = re.sub("how's", 'how is', word)
    word = re.sub("who's", 'who is', word)
    word = re.sub("why's", 'why is', word)
    word = re.sub("he's", 'he is', word)
    word = re.sub("it's", 'it is', word)
    word = re.sub("she's", 'she is', word)
    word = re.sub("\'s", 'possession', word)
    word = re.sub("eg", 'example', word)
    word = re.sub("usa", 'america', word)
    word = re.sub("u.s.a", 'america', word)
    word = re.sub("\'ll", 'will', word)
    word = re.sub("uk", 'england', word)
    word = re.sub("911", 'emergency', word)
    return word


  # removing punctuation function
  punct =[]
  punct += list(string.punctuation)
  punct += '’'
  punct.remove("'")
  def remove_punctuations(text):
      for punctuation in punct:
          text = text.replace(punctuation, ' ')
      return text

  # tokenization function
  from nltk import word_tokenize

  df_input = df
  df_input['cleaned'] = df_input[raw].apply(clean_word)
  # getting rid of whitespaces
  df_input['cleaned'] = df_input[raw].apply(lambda x: str(x).replace('\n', ' '))
  # remove links
  df_input['cleaned'] = df_input['cleaned'].str.replace('http\S+|www.\S+', '', case=False)
  # removing '>'
  df_input['cleaned'] = df_input['cleaned'].apply(lambda x: x.replace('&gt;', ''))
  # removing '<'
  df_input['cleaned'] = df_input['cleaned'].apply(lambda x: x.replace('&lt;', ''))
  # remove punctuation
  df_input['cleaned'] = df_input['cleaned'].apply(remove_punctuations)
  # remove ' s ' that was created after removing punctuations
  df_input['cleaned'] = df_input['cleaned'].apply(lambda x: str(x).replace(" s ", " "))
  # apply tokenization
  df_input['tokenized'] = df_input['cleaned'].apply(word_tokenize)
  # remove stop words
  df_input['tokenized'] = df_input['tokenized'].apply(lambda word_list: [x for x in word_list if x not in ENGLISH_STOP_WORDS])
  # remove numbers
  df_input['tokenized'] = df_input['tokenized'].apply(lambda list_data: [x for x in list_data if x.isalpha()])
  # finalizing the preprocessing
  finale = list()
  for list_of_words in df_input.tokenized:
    finale.append(' '.join(list_of_words))

  df_input['final_text'] = finale

  return df_input.tokenized

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
quora = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Quora Question Pairs/train.csv')
questions_df = quora[['question1', 'question2', 'is_duplicate']]
y = questions_df['is_duplicate']
q1 = questions_df[['question1']]
q2 = questions_df[['question2']]

# preprocess question1
question1_word_tokenized = preprocess_question(q1, 'question1')
question2_word_tokenized = preprocess_question(q2, 'question2')

In [ ]:
q1_v = list()
q2_v = list()
vocabulary = dict()
inverse_vocabulary = ['<placeholder>']
for row_q1, row_q2 in zip(question1_word_tokenized, question2_word_tokenized):
  q1_number = []
  q2_number = []
  for word1, word2 in zip(row_q1, row_q2):
    if word1 not in vocabulary:
      vocabulary[word1] = len(inverse_vocabulary)
      q1_number.append(len(inverse_vocabulary))
      inverse_vocabulary.append(word1)
    else:
      q1_number.append(vocabulary[word1])
    
    if word2 not in vocabulary:
      vocabulary[word2] = len(inverse_vocabulary)
      q2_number.append(len(inverse_vocabulary))
      inverse_vocabulary.append(word2)
    else:
      q2_number.append(vocabulary[word2])
  q1_v.append(q1_number)
  q2_v.append(q2_number)

In [ ]:
pd.DataFrame(zip(q1_v, q2_v))

,0,1
0,"[1, 2, 2, 3, 4, 5, 6]","[1, 2, 2, 3, 4, 5, 6]"
1,"[1, 7, 9, 11, 13, 15]","[1, 8, 10, 12, 14, 9]"
2,"[16, 17, 19, 20, 22, 24]","[16, 18, 20, 21, 23, 25]"
3,"[26, 17, 29, 31, 16]","[27, 28, 30, 30, 32]"
4,"[33, 34, 36, 38, 40]","[33, 35, 37, 39, 36]"
...,...,...
404285,"[16, 3116, 76526, 379, 382, 3123, 3124]","[16, 3116, 74640, 3121, 3122, 3123, 3124]"
404286,"[327, 2022, 443, 1474]","[263, 958, 443, 1474]"
404287,"[1, 10376]","[1, 10376]"
404288,"[1, 21907, 12041, 397, 2316, 2650, 57377, 8186...","[17, 837, 2421, 2398, 2194, 17, 120, 63, 730, ..."


In [ ]:
import gensim.downloader as api
model = api.load("glove-wiki-gigaword-300")
embedding_dim = 300

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
embeddings = 1 * np.random.randn(len(vocabulary) + 1, 300)  
embeddings[0] = 0

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in model.vocab:
        embeddings[index] = model.word_vec(word)

In [ ]:
x = pd.DataFrame(zip(q1_v,q2_v))
x = x.rename({0:'q1', 1:'q2'}, axis = 'columns')
x

,q1,q2
0,"[1, 2, 2, 3, 4, 5, 6]","[1, 2, 2, 3, 4, 5, 6]"
1,"[1, 7, 9, 11, 13, 15]","[1, 8, 10, 12, 14, 9]"
2,"[16, 17, 19, 20, 22, 24]","[16, 18, 20, 21, 23, 25]"
3,"[26, 17, 29, 31, 16]","[27, 28, 30, 30, 32]"
4,"[33, 34, 36, 38, 40]","[33, 35, 37, 39, 36]"
...,...,...
404285,"[16, 3116, 76526, 379, 382, 3123, 3124]","[16, 3116, 74640, 3121, 3122, 3123, 3124]"
404286,"[327, 2022, 443, 1474]","[263, 958, 443, 1474]"
404287,"[1, 10376]","[1, 10376]"
404288,"[1, 21907, 12041, 397, 2316, 2650, 57377, 8186...","[17, 837, 2421, 2398, 2194, 17, 120, 63, 730, ..."


In [ ]:
# finding out maximum length of question
max_length = max(x.q1.map(lambda x : len(x)).max(),
                 x.q2.map(lambda x : len(x)).max())

# splitting data to train test split
test_size = 40000
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)

# creating q1 and q2 partition
x_train = {'q1': x_train.q1, 'q2':x_train.q2}
x_test = {'q1':x_test.q1, 'q2':x_test.q2}

# convert y to numpy array
y_train = y_train.values
y_test = y_test.values

# zero padding
for data, part in itertools.product([x_train, x_test], ['q1', 'q2']):
  data[part] = tf.keras.preprocessing.sequence.pad_sequences(data[part], maxlen=max_length)

assert x_train['q1'].shape == x_train['q2'].shape

if len(x_train['q1']) == len(y_train):
  print('Data preparation complete. We can begin training now.')
else:
  print('Train data and label has different length.')

Data preparation complete. We can begin training now.


In [ ]:
import tensorflow.keras.backend as k

In [ ]:
!pip install keras_self_attention

  Created wheel for keras-self-attention: filename=keras_self_attention-0.47.0-cp36-none-any.whl size=17289 sha256=78c936d95acda298be8dbaaaba68127b1ae413c8d143124e214c7fa4778f0da3
  Stored in directory: /root/.cache/pip/wheels/70/87/01/76c703d5401b65e323927c1fdc665f3fb143282ff67d71e859
Successfully built keras-self-attention


In [ ]:
# Model variables
k.clear_session()
batch_size = 128
n_epoch = 10

q1_input = tf.keras.layers.Input(shape = (max_length, ), dtype = 'int32')
q2_input = tf.keras.layers.Input(shape = (max_length, ), dtype = 'int32')

embedding_layer = tf.keras.layers.Embedding(len(embeddings), embedding_dim, 
                                            weights = [embeddings], 
                                            input_length = max_length,
                                            trainable = False)

q1_embed = embedding_layer(q1_input)
q2_embed = embedding_layer(q2_input)

lstm_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences = True))

q1_lstm = lstm_layer(q1_embed)
q2_lstm = lstm_layer(q2_embed)

dot_layer = tf.keras.layers.dot([q1_lstm,q2_lstm], [1,1], normalize = True)
flatten_layer = tf.keras.layers.Flatten()(dot_layer)

bn_layer_1 = tf.keras.layers.BatchNormalization()(flatten_layer)
dn_layer_1 = tf.keras.layers.Dense(256, activation = 'selu')(bn_layer_1)
do_layer_1 = tf.keras.layers.Dropout(0.3)(dn_layer_1)

bn_layer_2 = tf.keras.layers.BatchNormalization()(do_layer_1)
dn_layer_2 = tf.keras.layers.Dense(256, activation = 'selu')(bn_layer_2)
do_layer_2 = tf.keras.layers.Dropout(0.3)(dn_layer_2)

bn_layer_3 = tf.keras.layers.BatchNormalization()(do_layer_2)
dn_layer_3 = tf.keras.layers.Dense(256, activation = 'selu')(bn_layer_3)
do_layer_3 = tf.keras.layers.Dropout(0.3)(dn_layer_3)


#distance_layer = tf.keras.layers.Lambda(function = lambda x: manhattan_distance(x[0],x[1]), 
#                                       output_shape = lambda x: x[0], name = 'distance')([q1_lstm, q2_lstm])

dense_predict = tf.keras.layers.Dense(1, activation = 'sigmoid')(do_layer_3)
full_model = tf.keras.models.Model([q1_input, q2_input], dense_predict)

full_model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = 'adam', metrics = ['accuracy'])
history = full_model.fit([x_train['q1'], x_train['q2']], y_train, batch_size=batch_size, epochs = n_epoch,
                         validation_data = ([x_test['q1'], x_test['q2']], y_test))


Epoch 1/10
2847/2847 [==============================] - 105s 37ms/step - loss: 0.5068 - accuracy: 0.7443 - val_loss: 0.4646 - val_accuracy: 0.7658
Epoch 2/10
2847/2847 [==============================] - 106s 37ms/step - loss: 0.4192 - accuracy: 0.7954 - val_loss: 0.4325 - val_accuracy: 0.7908
Epoch 3/10
2847/2847 [==============================] - 106s 37ms/step - loss: 0.3483 - accuracy: 0.8364 - val_loss: 0.4321 - val_accuracy: 0.7995
Epoch 4/10
2847/2847 [==============================] - 106s 37ms/step - loss: 0.2725 - accuracy: 0.8764 - val_loss: 0.4854 - val_accuracy: 0.7981
Epoch 5/10
2847/2847 [==============================] - 106s 37ms/step - loss: 0.2137 - accuracy: 0.9064 - val_loss: 0.5002 - val_accuracy: 0.8005
Epoch 6/10
2847/2847 [==============================] - 106s 37ms/step - loss: 0.1742 - accuracy: 0.9251 - val_loss: 0.4859 - val_accuracy: 0.8027
Epoch 7/10
2847/2847 [==============================] - 106s 37ms/step - loss: 0.1502 - accuracy: 0.9370 - val_loss: 0

In [ ]:
full_model.save_weights('siamese_full_model_weights.h5')